# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [56]:
import numpy as np
import pandas as pd
import xlsxwriter
import math
import requests
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [57]:
stocks = pd.read_csv('sp_500_stocks.csv')
from iextoken import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [58]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [59]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [60]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [61]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_df = final_df.append(
                pd.Series(
                    [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                    ],
                index = my_columns
            ),
            ignore_index = True
            )
        except:
            print(f'No data found for {symbol}')
final_df = final_df.dropna(subset=['Price-to-Earnings Ratio'])
final_df

C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The

No data found for ANR
No data found for ABC
No data found for BHI
No data found for CBG


C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The

No data found for COH
No data found for CCE
No data found for CSC
No data found for DISCA
No data found for DPS
No data found for FII


C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The

No data found for HCN
No data found for JEC
No data found for JDSU
No data found for KFT
No data found for LUK
No data found for LTD
No data found for MHP
No data found for PCS
No data found for PCLN


C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The

No data found for SYMC
No data found for TSO
No data found for TYC
No data found for WAG
No data found for WPO
No data found for WPI
No data found for WLP
No data found for WYN
No data found for YHOO
No data found for ZMH


C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\1005131813.py:8: FutureWarning: The

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,97.04,-36.9,N/A
1,ABT,98.15,33.38,N/A
2,ANF,50.63,24.22,N/A
3,ACN,316.09,28.2,N/A
4,ADBE,512.90,48.99,N/A
...,...,...,...,...
464,XRX,16.09,-9.46,N/A
465,XLNX,194.92,52.54,N/A
467,XYL,90.90,48.35,N/A
468,YUM,125.00,25.25,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [62]:
final_df.sort_values('Price-to-Earnings Ratio', inplace=True)
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df = final_df[:50]
final_df.reset_index(inplace=True)
final_df.drop('index', axis=1, inplace=True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,360.160,0.01,N/A
1,DNR,0.241,0.35,N/A
2,SWN,6.090,1.22,N/A
3,CNX,21.350,2,N/A
4,BTU,23.000,2.57,N/A
5,ZION,34.100,3.54,N/A
6,M,10.870,4.03,N/A
7,RRD,10.840,4.06,N/A
8,EMR,97.100,4.28,N/A
9,CMA,41.220,4.32,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [63]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [64]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [65]:
position_size = float(portfolio_size) / len(final_df.index)

for row in final_df.index:
    final_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[row, 'Price'])
    
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,360.160,0.01,55
1,DNR,0.241,0.35,82987
2,SWN,6.090,1.22,3284
3,CNX,21.350,2,936
4,BTU,23.000,2.57,869
5,ZION,34.100,3.54,586
6,M,10.870,4.03,1839
7,RRD,10.840,4.06,1845
8,EMR,97.100,4.28,205
9,CMA,41.220,4.32,485


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [66]:
symbol = 'AAPL'
batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_url).json()
print(data[symbol]['advanced-stats'])
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

{'beta': 1.3271866325627353, 'totalCash': 62482000000, 'currentDebt': 124963000000, 'revenue': 383933000000, 'grossProfit': 166816000000, 'totalRevenue': 383933000000, 'EBITDA': 123957000000, 'revenuePerShare': 24.56, 'revenuePerEmployee': 2611789.12, 'debtToEquity': 5.558582473371603, 'profitMargin': 0.2468138972164414, 'enterpriseValue': 2779505411280, 'enterpriseValueToRevenue': 7.24, 'priceToSales': 7.12, 'priceToBook': 45.34, 'forwardPERatio': 29.299754313218912, 'pegRatio': -16.766515726802137, 'peHigh': 40.13439229376595, 'peLow': 16.201973230903462, 'week52highDate': '2023-07-19', 'week52lowDate': '2023-01-03', 'putCallRatio': 0.5166976347109928, 'companyName': 'Apple Inc', 'marketcap': 2732707411280, 'week52high': 197.7, 'week52low': 123.48, 'week52highSplitAdjustOnly': 198.23, 'week52highDateSplitAdjustOnly': '2023-07-19', 'week52lowSplitAdjustOnly': 124.17, 'week52lowDateSplitAdjustOnly': '2023-01-03', 'week52change': 0.1718846901401181, 'sharesOutstanding': 15634232000, 'fl

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [67]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price to Earnings Ratio', 
    'PE Percentile',
    'Price to Book Ratio',
    'PB Percentile', 
    'Price to Sales Ratio', 
    'PS Percentile', 
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try:
            rv_df = rv_df.append(
                pd.Series(
                    [
                        symbol, 
                        data[symbol]['quote']['latestPrice'],
                        'N/A',
                        data[symbol]['quote']['peRatio'],
                        'N/A',
                        data[symbol]['advanced-stats']['priceToBook'],
                        'N/A',
                        data[symbol]['advanced-stats']['priceToSales'],
                        'N/A',
                        data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA'],
                        'N/A',
                        data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit'],
                        'N/A',
                        'N/A'
                    ],
                index = rv_columns
                ),
                ignore_index = True
        )
        except:
            pass

rv_df
     


C:\Users\user\AppData\Local\Temp\ipykernel_17300\2010197291.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\2010197291.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\2010197291.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\2010197291.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\user\AppData\Local\Temp\ipykernel_17300\2010197291.py:25: FutureWarning: The frame.append metho

,Ticker,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,97.04,N/A,-36.90,N/A,6.8700,N/A,1.620,N/A,-18.418505,N/A,4.492380,N/A,N/A
1,ABT,98.15,N/A,33.38,N/A,4.5800,N/A,4.230,N/A,22.078982,N/A,8.067435,N/A,N/A
2,ANF,50.63,N/A,24.22,N/A,3.3200,N/A,0.662,N/A,5.945221,N/A,0.976503,N/A,N/A
3,ACN,316.09,N/A,28.20,N/A,8.2900,N/A,3.300,N/A,16.888310,N/A,9.831005,N/A,N/A
4,ADBE,512.90,N/A,48.99,N/A,15.7600,N/A,12.690,N/A,36.245261,N/A,14.269541,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,XEL,59.73,N/A,18.96,N/A,1.9500,N/A,2.160,N/A,11.986436,N/A,6.986938,N/A,N/A
366,XRX,16.09,N/A,-9.46,N/A,0.7471,N/A,0.353,N/A,5.472471,N/A,1.010633,N/A,N/A
367,XYL,90.90,N/A,48.35,N/A,2.2100,N/A,3.610,N/A,25.122283,N/A,10.327076,N/A,N/A
368,YUM,125.00,N/A,25.25,N/A,-4.1500,N/A,5.010,N/A,19.574813,N/A,8.800269,N/A,N/A


In [68]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [71]:
from scipy.stats import percentileofscore as score

metrics = {
            'Price to Earnings Ratio': 'PE Percentile',
            'Price to Book Ratio': 'PB Percentile',
            'Price to Sales Ratio': 'PS Percentile',
            'EV/EBITDA': 'EV/EBITDA Percentile',
            'EV/GP': 'EV/GP Percentile'
        }

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = score(rv_df[metric], rv_df.loc[row, metric])/100

rv_df


,Ticker,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,97.04,N/A,-36.90,0.035135,6.8700,0.816216,1.620,0.439189,-18.418505,0.013514,4.492380,0.402703,N/A
1,ABT,98.15,N/A,33.38,0.840541,4.5800,0.708108,4.230,0.786486,22.078982,0.886486,8.067435,0.7,N/A
2,ANF,50.63,N/A,24.22,0.672973,3.3200,0.597297,0.662,0.148649,5.945221,0.191892,0.976503,0.064865,N/A
3,ACN,316.09,N/A,28.20,0.737838,8.2900,0.864865,3.300,0.716216,16.888310,0.748649,9.831005,0.8,N/A
4,ADBE,512.90,N/A,48.99,0.910811,15.7600,0.935135,12.690,0.978378,36.245261,0.967568,14.269541,0.943243,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,XEL,59.73,N/A,18.96,0.555405,1.9500,0.386486,2.160,0.582432,11.986436,0.505405,6.986938,0.624324,N/A
366,XRX,16.09,N/A,-9.46,0.078378,0.7471,0.094595,0.353,0.072973,5.472471,0.162162,1.010633,0.07027,N/A
367,XYL,90.90,N/A,48.35,0.908108,2.2100,0.441892,3.610,0.743243,25.122283,0.921622,10.327076,0.824324,N/A
368,YUM,125.00,N/A,25.25,0.694595,-4.1500,0.054054,5.010,0.843243,19.574813,0.827027,8.800269,0.745946,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [73]:
from statistics import mean

for row in rv_df.index:
    rv_df.loc[row, 'RV Score'] = mean([rv_df.loc[row, metrics[metric]] for metric in metrics.keys()])

rv_df


,Ticker,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,97.04,N/A,-36.90,0.035135,6.8700,0.816216,1.620,0.439189,-18.418505,0.013514,4.492380,0.402703,0.341351
1,ABT,98.15,N/A,33.38,0.840541,4.5800,0.708108,4.230,0.786486,22.078982,0.886486,8.067435,0.7,0.784324
2,ANF,50.63,N/A,24.22,0.672973,3.3200,0.597297,0.662,0.148649,5.945221,0.191892,0.976503,0.064865,0.335135
3,ACN,316.09,N/A,28.20,0.737838,8.2900,0.864865,3.300,0.716216,16.888310,0.748649,9.831005,0.8,0.773514
4,ADBE,512.90,N/A,48.99,0.910811,15.7600,0.935135,12.690,0.978378,36.245261,0.967568,14.269541,0.943243,0.947027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,XEL,59.73,N/A,18.96,0.555405,1.9500,0.386486,2.160,0.582432,11.986436,0.505405,6.986938,0.624324,0.530811
366,XRX,16.09,N/A,-9.46,0.078378,0.7471,0.094595,0.353,0.072973,5.472471,0.162162,1.010633,0.07027,0.095676
367,XYL,90.90,N/A,48.35,0.908108,2.2100,0.441892,3.610,0.743243,25.122283,0.921622,10.327076,0.824324,0.767838
368,YUM,125.00,N/A,25.25,0.694595,-4.1500,0.054054,5.010,0.843243,19.574813,0.827027,8.800269,0.745946,0.632973


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [74]:
rv_df.sort_values('RV Score', ascending=True, inplace=True)
rv_df = rv_df[:50]
rv_df.reset_index(drop=True, inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FOSL,2.00,N/A,-1.46,0.116216,0.3091,0.062162,0.0663,0.005405,-16.342658,0.016216,0.280030,0.005405,0.041081
1,BIG,4.93,N/A,-0.25,0.124324,0.4703,0.078378,0.0287,0.002703,-1.624058,0.02973,0.339961,0.008108,0.048649
2,LNC,24.65,N/A,-1.39,0.121622,0.8652,0.110811,0.2720,0.045946,-1.426673,0.032432,0.272008,0.002703,0.062703
3,M,10.87,N/A,4.03,0.143243,0.7065,0.091892,0.1218,0.010811,2.650098,0.048649,0.576516,0.024324,0.063784
4,GNW,5.92,N/A,6.16,0.181081,0.3675,0.064865,0.3665,0.075676,2.569332,0.043243,0.366508,0.010811,0.075135
5,BTU,23.00,N/A,2.57,0.137838,0.8867,0.121622,0.5453,0.124324,1.138545,0.035135,0.412111,0.013514,0.086486
6,XRX,16.09,N/A,-9.46,0.078378,0.7471,0.094595,0.3530,0.072973,5.472471,0.162162,1.010633,0.07027,0.095676
7,KSS,20.58,N/A,-22.02,0.051351,0.6099,0.081081,0.1282,0.013514,7.696230,0.281081,1.109121,0.083784,0.102162
8,C,40.92,N/A,6.52,0.191892,0.4227,0.07027,0.6013,0.143243,3.585914,0.081081,0.634431,0.027027,0.102703
9,SWN,6.09,N/A,1.22,0.132432,1.0300,0.156757,0.5932,0.135135,1.872299,0.040541,0.947905,0.059459,0.104865


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [75]:
portfolio_input()

In [79]:
position_size = float(portfolio_size)/len(rv_df.index)

for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_df.loc[row, 'Price'])
rv_df

,Ticker,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FOSL,2.00,25000,-1.46,0.116216,0.3091,0.062162,0.0663,0.005405,-16.342658,0.016216,0.280030,0.005405,0.041081
1,BIG,4.93,10141,-0.25,0.124324,0.4703,0.078378,0.0287,0.002703,-1.624058,0.02973,0.339961,0.008108,0.048649
2,LNC,24.65,2028,-1.39,0.121622,0.8652,0.110811,0.2720,0.045946,-1.426673,0.032432,0.272008,0.002703,0.062703
3,M,10.87,4599,4.03,0.143243,0.7065,0.091892,0.1218,0.010811,2.650098,0.048649,0.576516,0.024324,0.063784
4,GNW,5.92,8445,6.16,0.181081,0.3675,0.064865,0.3665,0.075676,2.569332,0.043243,0.366508,0.010811,0.075135
5,BTU,23.00,2173,2.57,0.137838,0.8867,0.121622,0.5453,0.124324,1.138545,0.035135,0.412111,0.013514,0.086486
6,XRX,16.09,3107,-9.46,0.078378,0.7471,0.094595,0.3530,0.072973,5.472471,0.162162,1.010633,0.07027,0.095676
7,KSS,20.58,2429,-22.02,0.051351,0.6099,0.081081,0.1282,0.013514,7.696230,0.281081,1.109121,0.083784,0.102162
8,C,40.92,1221,6.52,0.191892,0.4227,0.07027,0.6013,0.143243,3.585914,0.081081,0.634431,0.027027,0.102703
9,SWN,6.09,8210,1.22,0.132432,1.0300,0.156757,0.5932,0.135135,1.872299,0.040541,0.947905,0.059459,0.104865


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [80]:
writer = pd.ExcelWriter('Value Strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [82]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [84]:
column_formats = {
                    'A': ['Ticker', string_template],   
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price to Earnings Ratio', percent_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price to Book Ratio', percent_template],
                    'G': ['PB Percentile', percent_template],
                    'H': ['Price to Sales Ratio', percent_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', percent_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', percent_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [85]:
writer.save()

C:\Users\user\AppData\Local\Temp\ipykernel_17300\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
